<a href="https://colab.research.google.com/github/KrishnanandSingh/EVA/blob/master/Project-16/Assignment16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
tf.enable_eager_execution()

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
tf.keras.backend.set_session(tf.Session(config=config))

print('tf version: ', tf.version.VERSION, ', keras version: ', tf.keras.__version__)

tf version:  1.14.0 , keras version:  2.2.4-tf


In [0]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.optimizers import Adam

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

### download easyink library to get cifar tf records

In [0]:
!rm -rf easyink

!wget -q -O easyink-0.1.1.tar.gz https://github.com/inkers/easyink/archive/v0.1.1.tar.gz
!tar -xf easyink-0.1.1.tar.gz
!rm easyink-0.1.1.tar.gz
!mv easyink-0.1.1 easyink

In [0]:
!ls easyink/

datasets  example  LICENSE  README.md  visualise


In [0]:
from easyink.datasets import cifardata as cfdata
FLAGS = {
    'shuffle_buffer_size': 100,
    'num_parallel_reads': 3,
    'batch_size': batch_size
}
cifar = cfdata.Cifar()
cifar.generate_tfrecords()

train_dataset = cifar.get_train_dataset(FLAGS)
eval_dataset = cifar.get_eval_dataset(FLAGS)
val_dataset = eval_dataset
img_height, img_width, channel = 32, 32, 3

Generating ./train.tfrecords
Generating ./eval.tfrecords
Done!


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 12
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [0]:
model = Model(inputs=[input], outputs=[output])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 12)   324         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 12)   48          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 12)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
model.fit(train_dataset,
          epochs=epochs,
          steps_per_epoch=50000/batch_size,
          verbose=1,
          validation_data=val_dataset,
          validation_steps=1000/batch_size
          )

Epoch 1/50
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
391/390 [==============================] - 133s 341ms/step - loss: 1.7375 - acc: 0.3393 - val_loss: 1.7469 - val_acc: 0.3770
Epoch 2/50
391/390 [==============================] - 69s 176ms/step - loss: 1.3823 - acc: 0.4851 - val_loss: 1.4069 - val_acc: 0.5098
Epoch 3/50
391/390 [==============================] - 69s 176ms/step - loss: 1.2096 - acc: 0.5566 - val_loss: 1.4292 - val_acc: 0.5059
Epoch 4/50
391/390 [==============================] - 69s 176ms/step - loss: 1.0914 - acc: 0.6020 - val_loss: 1.3786 - val_acc: 0.5537
Epoch 5/50
391/390 [==============================] - 69s 177ms/step - loss: 1.0016 - acc: 0.6359 - val_loss: 1.3633 - val_acc: 0.5498
Epoch 6/50
391/390 [==============================] - 69s 176ms/step - loss: 0.9234 - acc: 0.6645 - val_loss: 1.2614 - val_acc: 0.5801
Epoch 7/50
391/390 [==============================] - 69s 176ms/step - loss: 0.8669 - acc: 0.68

In [0]:
score = model.evaluate_generator(eval_dataset, steps=1000/batch_size, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

8/7 [==============================] - 0s 43ms/step - loss: 1.5336 - acc: 0.6846
Test loss: 1.5662066192626953
Test accuracy: 0.6845703


In [0]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk
